Module importieren

In [98]:
import matplotlib.pyplot as plt
import pandas as pd
#import pd as pd
import seaborn as sns
from datetime import datetime
from fitter import Fitter
import numpy as np
from scipy import stats
import statsmodels.api as sm
from tabulate import tabulate

Daten laden und säubern

In [99]:
def open_csv_to_df(directory):
    df = pd.read_csv(directory)
    df = df.iloc[0:,:4]
    df.columns = ["Date", "Time", "Mass_(kg)", "Velocity_(m/s)"]
    df.insert(4, "Energy_(kJ)", "")
    df.insert(5, "Date_and_Time", "")
    df.insert(6, "Time_delta_(h)", "")
    for i in range(len(df)):
        df.iloc[i,4] = 0.5 * float(df.iloc[i,2]) * float(df.iloc[i,3]) ** 2 / 1000 #calculation of Energy_(kJ)
    df = df.sort_values(['Date', 'Time']).reset_index(drop=True)
    df = df.dropna()

    for i in range(len(df)):
        date_raw = df.iloc[i,:].Date +" " +df.iloc[i,:].Time
        date1 = datetime.strptime(date_raw, '%Y-%m-%d %H:%M')
        df.iloc[i,5] = date1

    for i in range(len(df)-1):
        date1 = df.iloc[i,5]
        date2 = df.iloc[i+1,5]
        time_delta = date2 - date1
        time_delta = (time_delta.days*24) + (time_delta.seconds//3600)
        df.iloc[i+1,6] = time_delta

    #Fitter kann keine NAs handhaben aus diesem Grund wird erste Beobachtung mit dem Median der Zeitabstände ersetzt.
    df.iloc[0,6] = 0
    df['Time_delta_(h)'] = df['Time_delta_(h)'].astype('int')
    df.iloc[0,6] = df["Time_delta_(h)"].median()

    return df #this is the table with all stones combined

In [100]:
df1 = open_csv_to_df("./out_1.csv")
df2 = open_csv_to_df("./out_2.csv")

Funktion für Fitting von Verteilungen

In [101]:
def fit_distribution(data):
    count = 0
    fitted_mass = Fitter(data)
    fitted_mass.fit()
    ks_summary = fitted_mass.summary()
    distributions = ks_summary.index.values
    print(ks_summary.iloc[:,-2:])
    fig, ax = plt.subplots(1,5,figsize=(25, 5))
    table = [['Distribution', 'Parameters']]

    for i in distributions:
        param = fitted_mass.fitted_param[i]
        table.append([i, param])
        dist_eval = eval('stats.' + i + '.rvs(*param, size = 1000)')
        sm.qqplot_2samples(data, dist_eval, xlabel = i.capitalize() + ' Distribution', ylabel = 'Sample Distribution', line = '45' ,ax = ax[count])
        count += 1
    plt.show()
    print(tabulate(table, headers = 'firstrow'))

Zone 1

In [102]:
#fit_distribution(df1['Mass_(kg)'])

In [103]:
#fit_distribution(df1["Velocity_(m/s)"])

In [104]:
#fit_distribution(df1["Time_delta_(h)"])

Zone 2

In [105]:
#fit_distribution(df2['Mass_(kg)'])

In [106]:
#fit_distribution(df2["Velocity_(m/s)"])

In [107]:
#fit_distribution(df2["Time_delta_(h)"])

Monte Carlo

In [108]:
sim_count = 3_000_000


Zone 1

In [109]:
# Masse
param = stats.gamma.fit(df1['Mass_(kg)'])
zone1_mass = stats.gamma.rvs(*param, size = sim_count)

# Geschwindigkeit
param = stats.norm.fit(df1["Velocity_(m/s)"])
zone1_velocity = stats.norm.rvs(*param, size = sim_count)

# Zeitabstand
param = stats.expon.fit(df1["Time_delta_(h)"])
zone1_time_delta = stats.expon.rvs(*param, size = sim_count)

Zone 2

In [110]:
# Masse
param = stats.gamma.fit(df2['Mass_(kg)'])
zone2_mass = stats.gamma.rvs(*param, size = sim_count)

# Geschwindigkeit
param = stats.norm.fit(df2["Velocity_(m/s)"])
zone2_velocity = stats.norm.rvs(*param, size = sim_count)

# Zeitabstand
param = stats.expon.fit(df2["Time_delta_(h)"])
zone2_time_delta = stats.expon.rvs(*param, size = sim_count)

In [111]:
df1_sim = pd.DataFrame()
df1_sim.insert(0, "Mass_(kg)", zone1_mass)
df1_sim.insert(1, "Velocity_(m/s)", zone1_velocity)
df1_sim.insert(2, "Energy_(kJ)", "")
df1_sim.insert(3, "Time_delta_(h)", zone1_time_delta)


df1_sim["Time_delta_(h)"] = zone1_time_delta.round(4)
df1_sim["Time_delta_(h)"][0] = 0

df1_sim = df1_sim.dropna()
df1_sim["Energy_(kJ)"] = (0.5 * df1_sim["Mass_(kg)"] * (df1_sim["Velocity_(m/s)"] ** 2)) / 1000
df1_sim

C:\Users\logan\AppData\Local\Temp\ipykernel_16596\3105050015.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1_sim["Time_delta_(h)"][0] = 0


,Mass_(kg),Velocity_(m/s),Energy_(kJ),Time_delta_(h)
0,140.172439,9.800095,6.731211,0.0000
1,327.024189,9.799082,15.700761,60.3114
2,440.544796,10.111763,22.522359,24.0295
3,2300.664412,6.552020,49.382575,3.2734
4,244.405459,7.538445,6.944556,15.3464
...,...,...,...,...
2999995,457.721467,9.173828,19.260718,35.0983
2999996,778.989987,5.867912,13.411242,2.9859
2999997,1294.179470,6.667975,28.770831,3.4917
2999998,744.735843,7.969168,23.648210,66.9229


In [112]:
df2_sim = pd.DataFrame()
df2_sim.insert(0, "Mass_(kg)", zone2_mass)
df2_sim.insert(1, "Velocity_(m/s)", zone2_velocity)
df2_sim.insert(2, "Energy_(kJ)", "")
df2_sim.insert(3, "Time_delta_(h)", zone2_time_delta)


df2_sim["Time_delta_(h)"] = zone2_time_delta.round(4)
df2_sim["Time_delta_(h)"][0] = 0

df2_sim = df2_sim.dropna()
df2_sim["Energy_(kJ)"] = (0.5 * df2_sim["Mass_(kg)"] * (df2_sim["Velocity_(m/s)"] ** 2)) / 1000
df2_sim

C:\Users\logan\AppData\Local\Temp\ipykernel_16596\2961706192.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_sim["Time_delta_(h)"][0] = 0


,Mass_(kg),Velocity_(m/s),Energy_(kJ),Time_delta_(h)
0,140.579793,40.385488,114.641955,0.0000
1,8.216786,29.014936,3.458718,132.9791
2,622.853142,29.235832,266.186831,159.1987
3,60.906706,31.853373,30.899110,57.1034
4,35.673134,46.328779,38.283619,21.2805
...,...,...,...,...
2999995,159.271567,43.455264,150.381026,138.7827
2999996,19.925296,42.052602,17.618159,44.1996
2999997,60.479417,41.171362,51.258757,16.8978
2999998,53.747228,44.352080,52.863274,117.2631


In [113]:
df_sim = pd.concat([df1_sim,df2_sim])
df_sim = df_sim.sample(frac=1,ignore_index=True)

In [114]:
def netztest(status_netz,gewicht_im_netz,aufprallenergie):

    if status_netz == True:
        return True

    elif gewicht_im_netz > 2000:
        if aufprallenergie > 500:
            return True
        else:
            return False

    elif gewicht_im_netz <= 2000:
        if aufprallenergie > 1000:
            return True
        else:
            return False
    else:
        return False

In [115]:
df_sim

,Mass_(kg),Velocity_(m/s),Energy_(kJ),Time_delta_(h)
0,33.532599,40.094526,26.953018,68.6482
1,2.515971,45.225210,2.572982,95.7884
2,2329.584139,8.801585,90.233994,21.4226
3,0.429724,39.105926,0.328583,173.2467
4,902.595238,9.027108,36.775646,17.7670
...,...,...,...,...
5999995,178.637367,39.368058,138.430027,100.1942
5999996,1764.255943,8.233860,59.805152,6.2397
5999997,366.614778,9.779988,17.533017,13.6689
5999998,437.720166,7.547225,12.466404,2.0080


In [123]:
max_delta = 24
current_delta = 0
stunden_total = 0
masse_im_netz = 0
status_netz = False
incident_steine = 0
steine_im_netz = 0
length = len(df_sim)

for i in range(int(length)):
    iterated_delta = df_sim["Time_delta_(h)"][i]
    iterated_energy = df_sim["Energy_(kJ)"][i]

    stunden_total += iterated_delta

    if (iterated_delta > max_delta):
        if status_netz:
            incident_steine += steine_im_netz
        steine_im_netz = 1
        status_netz = netztest(False,0,iterated_energy)
        masse_im_netz = df_sim["Mass_(kg)"][i]

        iterated_delta = iterated_delta % max_delta


        if iterated_delta > (max_delta - current_delta):

            current_delta = abs((max_delta - current_delta)-iterated_delta)


        elif iterated_delta <= (max_delta - current_delta):

            current_delta = current_delta + iterated_delta

    elif iterated_delta <= max_delta:

        if iterated_delta > (max_delta - current_delta):
            if status_netz:
                incident_steine += steine_im_netz
            steine_im_netz = 1
            status_netz = netztest(False,0,iterated_energy)
            masse_im_netz = df_sim["Mass_(kg)"][i]
            current_delta = abs((max_delta - current_delta)-iterated_delta)

        elif iterated_delta <= (max_delta - current_delta):
            steine_im_netz += 1
            status_netz = netztest(status_netz,masse_im_netz,iterated_energy)
            masse_im_netz += df_sim["Mass_(kg)"][i]
            current_delta = current_delta + iterated_delta



print("Anzahl simulierte Steine: ",length)
print("Anzahl incident Steine: ",incident_steine)
print(incident_steine/length*100)

Anzahl simulierte Steine:  6000000
Anzahl incident Steine:  380
0.006333333333333333


In [124]:
anzahl_sim_jahre = stunden_total / 8760
steine_pro_jahr = (length / anzahl_sim_jahre)
incident_prob_pro_stein = incident_steine/length
incident_prob_pro_jahr = incident_prob_pro_stein * steine_pro_jahr
print(incident_prob_pro_jahr)



0.011538549064629063


In [153]:
def danger_zone(v_km_h,laenge_auto,autos_pro_tag):
    reaktionszeit = 0.1
    seconds_per_day = 86400
    v_m_s = (v_km_h / 3.6)
    danger_time_pro_auto = (laenge_auto / v_m_s)
    print(danger_time_pro_auto)
    danger_time_total = danger_time_pro_auto * autos_pro_tag
    print(danger_time_total)
    danger_percentage = ((danger_time_total / seconds_per_day))
    print(danger_percentage)

    return danger_percentage

In [168]:
danger_strecke = danger_zone(60,10.3,1200)

0.618
741.6
0.008583333333333333


In [169]:
print(incident_prob_pro_jahr)
print(danger_strecke)
print('{:f}'.format(incident_prob_pro_jahr * danger_strecke ))

0.011538549064629063
0.008583333333333333
0.000099
